1. Compare two cities and determine how similar or disimilar they are

2. In Washington D.C. if someone is looking to open a restaurant, where would you recommend that they open it? What about a Contractor?

## Week One

Graded by:

1. A description of the problem and a discussion of the background. (15 marks)
2. A description of the data and how it will be used to solve the problem. (15 marks)

## Week Two

Graded by:

1. A link to your Notebook on your Github repository, showing your code. (15 marks)
2. A full report consisting of all of the following components (15 marks):

Introduction where you discuss the business problem and who would be interested in this project.

Data where you describe the data that will be used to solve the problem and the source of the data.

Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.

Results section where you discuss the results.

Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.

Conclusion section where you conclude the report.

3. Your choice of a presentation or blogpost. (10 marks)



Best place to open a bar?
Metrics - stars, # of reviews, sentiment of reviews, population, population/#of bars(lower is better, less competition)
normalize, then multiply across to get a "bar estimate score" 
Use Linear Regression to see if score 

In [ ]:
### The purpose of this project is to find the best pizza 

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
# !conda install -c conda-forge folium=0.5.0 --yes
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from pandas.io.json import json_normalize




CLIENT_ID = 'CPZ3IZI3POLVKONVI1F1MA4AUFZEQEKG1HODGEKRXNFA1WLX' 
CLIENT_SECRET = '0Q1ZUIXMVFJIIRN221SSYBWYUSZIWZRJYJ1LUH4BNB2ROVMO' 
ACCESS_TOKEN = 'Q5XIKVYCSWCY5E0PMVZ3MKOXTD4BQ3WNQLORKNM0CXDM3YEN' 
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CPZ3IZI3POLVKONVI1F1MA4AUFZEQEKG1HODGEKRXNFA1WLX
CLIENT_SECRET:0Q1ZUIXMVFJIIRN221SSYBWYUSZIWZRJYJ1LUH4BNB2ROVMO


In [2]:
latitude = 40.7149555 
longitude = -74.0153365
search_query = 'Pizza'
radius = 500


url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=CPZ3IZI3POLVKONVI1F1MA4AUFZEQEKG1HODGEKRXNFA1WLX&client_secret=0Q1ZUIXMVFJIIRN221SSYBWYUSZIWZRJYJ1LUH4BNB2ROVMO&ll=40.7149555,-74.0153365&oauth_token=Q5XIKVYCSWCY5E0PMVZ3MKOXTD4BQ3WNQLORKNM0CXDM3YEN&v=20180604&query=Pizza&radius=500&limit=30'

In [3]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60be56465fb0fb2e9b06b839'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '57acc558498e036ce8f2b0b4',
    'name': 'La Pizza & La Pasta',
    'location': {'address': '4 World Trade Ctr',
     'crossStreet': 'Liberty St',
     'lat': 40.710566583744715,
     'lng': -74.01249396848024,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.710566583744715,
       'lng': -74.01249396848024}],
     'distance': 544,
     'postalCode': '10007',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['4 World Trade Ctr (Liberty St)',
      'New York, NY 10007']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primar

In [4]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/daniel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.city,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,57acc558498e036ce8f2b0b4,La Pizza & La Pasta,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1623086662,False,4 World Trade Ctr,Liberty St,40.710567,-74.012494,"[{'label': 'display', 'lat': 40.71056658374471...",...,New York,NY,United States,"[4 World Trade Ctr (Liberty St), New York, NY ...",NaN,NaN,NaN,NaN,NaN,NaN
1,4bc238adf8219c744286b410,Amore's Pizza Restaurant,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1623086662,False,147 Chambers St,Hudson Street,40.715860,-74.009888,"[{'label': 'display', 'lat': 40.71585960614924...",...,New York,NY,United States,"[147 Chambers St (Hudson Street), New York, NY...",1431324,https://www.seamless.com/menu/cafe-amores-pizz...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png
2,4c35ffa0a0ced13aa6aa1a6e,Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1623086662,False,154 Chambers St,NaN,40.715859,-74.009889,"[{'label': 'display', 'lat': 40.71585918421961...",...,New York,NY,United States,"[154 Chambers St, New York, NY 10007]",NaN,NaN,NaN,NaN,NaN,NaN
3,4fa862b3e4b0ebff2f749f06,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1623086662,False,225 Murray St,NaN,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",...,New York,NY,United States,"[225 Murray St, New York, NY 10282]",NaN,NaN,NaN,NaN,NaN,NaN
4,538df811498ed4e0c93f2dca,Skinny Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1623086662,False,225 Liberty St,NaN,40.712670,-74.015979,"[{'label': 'display', 'lat': 40.7126704467616,...",...,New York,NY,United States,"[225 Liberty St, New York, NY 10281]",NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,La Pizza & La Pasta,Pizza Place,4 World Trade Ctr,Liberty St,40.710567,-74.012494,"[{'label': 'display', 'lat': 40.71056658374471...",544,10007,US,New York,NY,United States,"[4 World Trade Ctr (Liberty St), New York, NY ...",57acc558498e036ce8f2b0b4
1,Amore's Pizza Restaurant,Pizza Place,147 Chambers St,Hudson Street,40.715860,-74.009888,"[{'label': 'display', 'lat': 40.71585960614924...",470,10007,US,New York,NY,United States,"[147 Chambers St (Hudson Street), New York, NY...",4bc238adf8219c744286b410
2,Pizza,Pizza Place,154 Chambers St,NaN,40.715859,-74.009889,"[{'label': 'display', 'lat': 40.71585918421961...",470,10007,US,New York,NY,United States,"[154 Chambers St, New York, NY 10007]",4c35ffa0a0ced13aa6aa1a6e
3,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,NaN,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282]",4fa862b3e4b0ebff2f749f06
4,Skinny Pizza,Pizza Place,225 Liberty St,NaN,40.712670,-74.015979,"[{'label': 'display', 'lat': 40.7126704467616,...",260,10281,US,New York,NY,United States,"[225 Liberty St, New York, NY 10281]",538df811498ed4e0c93f2dca
5,Pronto Pizza,Pizza Place,114 Liberty St,church,40.710018,-74.012418,"[{'label': 'display', 'lat': 40.71001795396908...",602,10006,US,New York,NY,United States,"[114 Liberty St (church), New York, NY 10006]",4bf6e92e5efe2d7fb1016834
6,Steack And Pizza,Pizza Place,NaN,NaN,40.716070,-74.010075,"[{'label': 'display', 'lat': 40.71607, 'lng': ...",460,NaN,US,New York,NY,United States,"[New York, NY]",51871910498ef4123991b93a
7,Brick Oven Pizza,Pizza Place,NaN,NaN,40.715817,-74.009898,"[{'label': 'display', 'lat': 40.71581695719298...",468,10007,US,New York,NY,United States,"[New York, NY 10007]",4c9a3413292a6dcbafdccc76
8,Tribeca Burger,Burger Joint,270 Greenwich St,NaN,40.715361,-74.011172,"[{'label': 'display', 'lat': 40.7153614563267,...",354,10007,US,New York,NY,United States,"[270 Greenwich St, New York, NY 10007]",5a5cf43c6a59504c346cb44d


In [5]:
venue_id = '57acc558498e036ce8f2b0b4' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url

result = requests.get(url).json()
print(result['response']['venue'].keys())
pd.DataFrame(result['response']['venue']['price'], index =[result['response']['venue']['name']])

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'seasonalHours', 'pageUpdates', 'inbox', 'parent', 'hierarchy', 'attributes', 'bestPhoto'])


,tier,message,currency
La Pizza & La Pasta,1,Cheap,$
